Installing tensorlfow for gpu

In [ ]:
!pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 18kB/s 
     |████████████████████████████████| 3.8MB 49.4MB/s 
     |████████████████████████████████| 450kB 58.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


importing required python libraries

In [ ]:
from google.colab import drive
import os
import sys
import h5py
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tables
from sklearn import preprocessing
import gdown
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, GlobalAveragePooling1D, GlobalMaxPooling1D
from sklearn.utils import class_weight

Setting data file paths and model weight files 

In [ ]:

FILT_FILE_PATH = 'TUH_filt.hdf5'

if not os.path.exists(FILT_FILE_PATH):
    gdown.download('https://drive.google.com/uc?id=1-NP_r_VxjR7np9n0tswqguTB5Vps0pO5', 
                   './'+FILE_PATH, quiet=False)


Defining function for getting data files and transforming into desired format for processing data into model


*   splitting data into train, validation and test dataset , train set 80 % , validation set = 10 % and Test Set = 10 %
*   Printing shape of the data for each sample set





In [ ]:

TEST_SIZE = 0.1
RANDOM_STATE = 0

def make_data(data_lists, data_type, filepath):
  data_dict = {}
  h5file = tables.open_file(filepath, mode="r+")
  for group_type in data_lists:
    key_list = data_lists[group_type]
    for i, part_id in enumerate(key_list):
      if i == 0:
        data = h5file.get_node('/'+part_id+'/'+data_type)[:]
      else:
        data = np.concatenate([data, h5file.get_node('/'+part_id+'/'+data_type)[:]])
    data_dict[group_type] = data
  return data_dict

def load_and_prep(FILE_PATH):
  f = h5py.File(FILE_PATH, 'r')
  key_list = list(f.keys())

  train_list, test_list = train_test_split(key_list, test_size=TEST_SIZE, 
                                          random_state=RANDOM_STATE)
  train_list, val_list = train_test_split(train_list, test_size=TEST_SIZE, 
                                          random_state=RANDOM_STATE)
  data_lists = {'train': train_list, 
                'test': test_list,
                'val': val_list}

  data_x_dict = make_data(data_lists, 'Data_x', FILE_PATH)
  X_train = data_x_dict['train']
  X_val = data_x_dict['val']
  X_test = data_x_dict['test']

  data_y_dict = make_data(data_lists, 'Data_y', FILE_PATH)
  y_train = data_y_dict['train']
  y_val = data_y_dict['val']
  y_test = data_y_dict['test']

  le = preprocessing.LabelEncoder()
  y_train = le.fit_transform(y_train)
  y_val = le.transform(y_val)
  y_test = le.transform(y_test)

  print(X_train.shape)
  print(X_val.shape)
  print(X_test.shape)

  print(y_train.shape)
  print(y_val.shape)
  print(y_test.shape)

  return X_train, X_val, X_test, y_train, y_val, y_test

X_train_filt, X_val_filt, X_test_filt, y_train_filt, y_val_filt, y_test_filt = load_and_prep(FILT_FILE_PATH)

(16764, 512, 22)
(2421, 512, 22)
(2814, 512, 22)
(16764,)
(2421,)
(2814,)


As we can see from above output, our dataframe is three dimensional.
different data windows (each with a class label in the y array)
a single data point (512 being the length of the window)
The various channels
As an example, let's take a look at one of the windows in one of the channels.

In [ ]:


clear_session()
model = Sequential()
# this just tells the model what input shape to expect
model.add(Input(shape=X_train_filt.shape[1:]))
for i in range(2):
  model.add(Conv1D(filters=64,
                  kernel_size=3,
                  padding="same",
                  activation='relu'))

In [ ]:

model.add(MaxPooling1D(pool_size=3, # size of the window
                       strides=2,   # factor to downsample
                       padding='same'))

In [ ]:
for i in range(2):
  model.add(Conv1D(filters=128,
                  kernel_size=3,
                  padding="same",
                  activation='relu'))

Adding flatten layer to the model

In [ ]:

model.add(Flatten())

 

*   Adding dense layer to the model.
*   Adding final dense layer with sigmoid as activation function.

*   Compiling the model
*   Printing summary of the model






In [ ]:


model.add(Dense(units=100,
                activation='relu'))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', 'AUC', 'Recall', 'Precision'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 512, 64)           4288      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 256, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 128)          24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 256, 128)          49280     
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               3

Defining compact function which do all of the following in one step


*   Took Train data sample as input 
*   Return final model after adding all the required layers


*   1D Convolution
*   Max pooling 

*   Fltten layer
*   Dense Layer

*   Compiling the model
*   Printing summary of the model


In [ ]:
# Returns a short sequential model
def create_model(input_shape, flatten=False):
  clear_session()
  model = Sequential()

  # this just tells the model what input shape to expect
  model.add(Input(shape=input_shape[1:]))
  for i in range(2):
    model.add(Conv1D(filters=64,
                    kernel_size=3,
                    padding="same",
                    activation='relu'))
    
  model.add(MaxPooling1D(pool_size=3, # size of the window
                       strides=2,   # factor to downsample
                       padding='same'))
  
  for i in range(2):
    model.add(Conv1D(filters=128,
                    kernel_size=3,
                    padding="same",
                    activation='relu'))
  if flatten:
    model.add(Flatten())
  else:
    model.add(GlobalAveragePooling1D())

  model.add(Dense(units=64,
                  activation='relu'))

  model.add(Dense(units=1,
                  activation='sigmoid'))

  model.compile(optimizer=Adam(0.001),
                loss='binary_crossentropy',
                metrics=['accuracy', 'AUC', 'Recall', 'Precision'])

  return model

clear_session()
# Create a basic model instance
model = create_model(X_train_filt.shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 512, 64)           4288      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 256, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 128)          24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 256, 128)          49280     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8

Defining call back functions for early stopping of the model to get best models weights during training

In [ ]:


def create_callbacks(best_model_filepath, tensorboard_logs_filepath):

  callback_checkpoint = ModelCheckpoint(filepath=best_model_filepath,
                                        monitor='val_loss',
                                        verbose=0,
                                        save_weights_only=True,
                                        save_best_only=True)
  
  callback_early_stopping = EarlyStopping(monitor='val_loss',
                                          patience=10, 
                                          verbose=1)
  
  callback_tensorboard = TensorBoard(log_dir=tensorboard_logs_filepath,
                                     histogram_freq=0,
                                     write_graph=False)
  
  callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.1,
                                         min_lr=1e-4,
                                         patience=0,
                                         verbose=1)
  
  return [callback_checkpoint, callback_early_stopping,
          callback_tensorboard, callback_reduce_lr]

Defining paths where model model weights needs to be stored

*   Defining hyperparameters like batch size, no. of epochs
*   Training the model



In [ ]:


EPOCHS = 20
BATCH_SIZE = 64
best_model_filepath = "CNN1D_Model.ckpt"
tensorboard_logs_filepath = "./CNN1D_logs/"

# calculate the class weights
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y_train_filt),
                                                  y_train_filt)

history_1D = model.fit(X_train_filt, 
                       y_train_filt,
                       batch_size=BATCH_SIZE, 
                       epochs=EPOCHS,
                       validation_data = (X_val_filt, y_val_filt),
                       callbacks= create_callbacks(best_model_filepath, 
                                                   tensorboard_logs_filepath),
                       class_weight = class_weights,
                       verbose=1)

Train on 16764 samples, validate on 2421 samples
Epoch 1/20
16764/16764 [==============================] - 4s 231us/sample - loss: 0.1140 - accuracy: 0.9772 - AUC: 0.8452 - Recall: 0.5566 - Precision: 0.7687 - val_loss: 0.0980 - val_accuracy: 0.9727 - val_AUC: 0.9726 - val_Recall: 0.7328 - val_Precision: 1.0000
Epoch 2/20
16764/16764 [==============================] - 3s 157us/sample - loss: 0.0464 - accuracy: 0.9880 - AUC: 0.9563 - Recall: 0.7608 - Precision: 0.9017 - val_loss: 0.0689 - val_accuracy: 0.9818 - val_AUC: 0.9767 - val_Recall: 0.8300 - val_Precision: 0.9903
Epoch 3/20
16448/16764 [============================>.] - ETA: 0s - loss: 0.0436 - accuracy: 0.9888 - AUC: 0.9636 - Recall: 0.7704 - Precision: 0.9167
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
16764/16764 [==============================] - 3s 162us/sample - loss: 0.0437 - accuracy: 0.9888 - AUC: 0.9640 - Recall: 0.7687 - Precision: 0.9181 - val_loss: 0.0822 - val_accuracy: 0.9777 -